<a href="https://colab.research.google.com/github/jwang44/Try-colabing-in-colab/blob/main/ECSE01_2_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [84]:
import numpy as np
import pandas as pd
import scipy.stats
import statistics

## The model

In [85]:
# sigmoid function
def sigmoid(a):
  return 1/(1+np.exp(-a))

In [138]:
class Logistic_regression():
  #def __init__(self):#,X_train,y_train,learning_rate,X_test,y_test):
    #self.X_train = X_train
    #self.y_train = y_train
    #self.learning_rate = learning_rate
    #self.X_test = X_test
    #self.y_test = y_test
    
  # training
  def fit(self,X_train,y_train,learning_rate):
    #n,m = np.shape(self.X_train)
    n,m = np.shape(X_train)  
    wk = np.ones([m+1,1]) # wk weights, initialized with 1
    wk1 = np.zeros([m+1,1])# wk+1 weights,initialized with 0          
    itrnum = 500       # max number of iterations 
    e = 0.001
    der = 0
    for k in range(0,itrnum):
      for i in range(0,n):
        #xi = self.X_train[i].T
        xi = X_train[i].T
        x0 = np.array([1])
        xi = np.concatenate((xi, x0),axis = 0)
        #yi = self.y_train[i]
        yi = y_train[i]
        der = der-xi*(yi-sigmoid(np.matmul(wk[:,0].T,xi))) # take derivative w.r.t w
      #wk1[:,0] = wk[:,0]-self.learning_rate*der       # update rule
      wk1[:,0] = wk[:,0]-learning_rate*der       # update rule
      if (np.linalg.norm(wk1[:,0]-wk[:,0]))**2<e:         
        break 
      else:
        wk = wk1.copy()
    return wk1
  
  # validation
  def predict(self,w,X_test):
    #n,m = np.shape(self.X_test)
    n,m = np.shape(X_test)   
    y_predict = np.zeros([n,1])
    for i in range(0,n):
      #xi = self.X_test[i].T
      xi = X_test[i].T
      x0 = np.array([1])
      xi = np.concatenate((xi, x0),axis = 0)
      p1 = sigmoid(np.matmul(w.T,xi)) # calculate probabilities p(y=1|x)
      # covert probabilities to 0 or 1 by thresholding at 0.5
      if p1>=0.5:
        y_predict[i] = 1
      else:
        y_predict[i] = 0
    return y_predict

  # evaluate accuracy
  def Accu_eval(self,y_test,y_predict):
    #y_predict = self.predict(X_test)
    n,j = np.shape(y_predict)
    TP = 0;FP = 0;TN = 0;FN = 0
    # count TP,TN,FP,FN in validation set
    '''for i in range(n):
      if  self.y_test[i]==1 and y_predict[i]==1:
        TP = TP+1
      elif self.y_test[i]==0 and y_predict[i]==0:
        TN = TN+1
      elif self.y_test[i]==0 and y_predict[i]==1:
        FP = FP+1
      elif self.y_test[i]==1 and y_predict[i]==0:
        FN = FN+1'''
    for i in range(n):
      if  y_test[i]==1 and y_predict[i]==1:
        TP = TP+1
      elif y_test[i]==0 and y_predict[i]==0:
        TN = TN+1
      elif y_test[i]==0 and y_predict[i]==1:
        FP = FP+1
      elif y_test[i]==1 and y_predict[i]==0:
        FN = FN+1    
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F = 2*precision*recall/(precision+recall)
    specificity = TN/(FP+TN)
    FPR = FP/(FP+TN)
    print("accuracy:",accuracy)
    print("precision:",precision)
    print("recall:",recall)
    print("F:",F)
    print("specificity:",specificity)
    print("False Positive Rate:",FPR)
    print("")
    return accuracy
    

## Cross validation

In [87]:
class Cross_validation():
  def __init__(self, k):
    # k: k-fold
    self.k = k

  def prepare_data(self, data):
    # data: np array converted from csv
    np.random.shuffle(data)
    X = data[:, :-1]  # features
    y = data[:, -1]   # labels

    # split data into k equal segments, assign them to train and test later
    Xs = np.array_split(X, self.k, axis=0)
    ys = np.array_split(y, self.k, axis=0)
    return Xs, ys

  def get_accuracy(self, Xs, ys, lr):
    accuracies = []
    for i in range(self.k):
      X_cv = Xs[:] # X_cross_validation
      y_cv = ys[:] # y_cross_validation

      X_test = X_cv.pop(i)
      y_test = y_cv.pop(i)

      X_train = np.concatenate(X_cv)
      y_train = np.concatenate(y_cv)

      logistic_regression = Logistic_regression(X_train, y_train, lr, X_test, y_test)

      print("----------FOLD ", i+1, "----------")
      accuracy = logistic_regression.Accu_eval()
      accuracies.append(accuracy)
    return np.mean(accuracies)
      # this will print the evaluation results

## Dataset

In [116]:
# generate new feature by multiplication and normalize
def newfeature(x,y):
  z=x*y
  norz=scipy.stats.zscore(z, axis=0, ddof=0, nan_policy='propagate')
  return norz

In [117]:
%cd "/content/drive/My Drive/"

/content/drive/My Drive


### Orthopedic Patients Dataset

In [118]:
# convert csv to dataframe
df = pd.read_csv('orthopedic_patients.csv')
original_data = df.to_numpy()

In [119]:
# normalize feature
NorData = scipy.stats.zscore(df.iloc[:,:-1], axis=0, ddof=0, nan_policy='propagate')
# normalized dataset
NorDataset = np.column_stack((NorData, df['Class']))
# np.savetxt('normalized_orthopedic_patients.csv', NorPatientData, delimiter=',')

New feature

In [120]:
# new feature
NewF = newfeature(df.pelvic_incidence, df.sacral_slope)
# normalized dataset with new feature
NorDatasetNew = np.column_stack((NorData,NewF,df['Class']))

### Credit Card Dataset

In [113]:
# convert csv to dataframe
df = pd.read_csv('creditcard.csv')
# data = df.to_numpy()

In [114]:
# normalize feature
NorData = scipy.stats.zscore(df.iloc[:,:-1], axis=0, ddof=0, nan_policy='propagate')
# normalized dataset
NorDataset = np.column_stack((NorData,df.iloc[:,-1]))

New feature

In [79]:
NewF1 = newfeature(df.V3, df.V7)
NewF2 = newfeature(df.V11, df.V12)
NewF3 = newfeature(df.V12, df.V16)
NewF4 = newfeature(df.V16, df.V17)
NewF5 = newfeature(df.V16, df.V18)
NewF6 = newfeature(df.V17, df.V18)
# new feature
NewF = np.column_stack((NewF1,NewF2,NewF3,NewF4,NewF5,NewF6))

In [121]:
# normalized dataset with new feature
NorDatasetNew = np.column_stack((NorData,NewF,df.iloc[:,-1]))

## Experiment with different features

In [122]:
lr = 0.001
cv = Cross_validation(10) # 10-fold cross-validation
Xs, ys = cv.prepare_data(NorDataset)
print("----------Using normalized features, without new features----------")
accu_avg = cv.get_accuracy(Xs, ys, lr)
print("----------AVERAGE ACCURACY", accu_avg, "----------")
print("\n---------------------------------------------------------------------")


----------Using normalized features, without new features----------
----------FOLD  1 ----------


ValueError: ignored

In [123]:
lr = 0.001
cv = Cross_validation(10) # 10-fold cross-validation
Xs, ys = cv.prepare_data(NorDatasetNew)
print("----------Using normalized features, with new features----------")
accu_avg = cv.get_accuracy(Xs, ys, lr)
print("----------AVERAGE ACCURACY", accu_avg, "----------")
print("\n---------------------------------------------------------------------")


----------Using normalized features, with new features----------
----------FOLD  1 ----------


ValueError: ignored

## Experiment with different learning rates

In [124]:
lrs = np.logspace(-6, -1, 6) # different learning rates to try
# or we can also try other hyperparameters here
cv = Cross_validation(10) # 10-fold cross-validation
Xs, ys = cv.prepare_data(original_data)
for lr in lrs:
  print("----------LEARNING RATE: ", lr, "----------")
  accu_avg = cv.get_accuracy(Xs, ys, lr)
  print("----------AVERAGE ACCURACY", accu_avg, "----------")
  print("\n---------------------------------------------------------------------\n")


----------LEARNING RATE:  1e-06 ----------
----------FOLD  1 ----------


ValueError: ignored